In [ ]:
import pandas as pd

def split_time_range(value):
    if pd.isna(value): 
        return pd.Series([None, None], index=['start', 'end'])
    if value in ['NONE', 'ALL']:
        return pd.Series([value, value], index=['start', 'end'])
    if ',' in str(value): 
        start, end = value.split(',', 1)
        return pd.Series([start.strip(), end.strip()], index=['start', 'end'])
    return pd.Series([value, value], index=['start', 'end'])

def get_parking_lots_single(df, time, parking_pass):
    pass_row = df[df.iloc[:, 0] == parking_pass]
    
    if pass_row.empty:
        return []
    
    available_lots = []
    for col in df.columns:
        if not (col.endswith('_start') or col.endswith('_end')):
            continue
        
        base_col = col.rsplit('_', 1)[0]
        
        start = pass_row[f'{base_col}_start'].values[0]
        end = pass_row[f'{base_col}_end'].values[0]
        
        if start == 'ALL' or end == 'ALL':
            available_lots.append(base_col) 
        elif start == 'NONE' or end == 'NONE':
            continue
        else:
            try:
                if (start == None) or (end == None):
                    continue
                start = int(start)
                end = int(end)
                if start <= time <= end:
                    available_lots.append(base_col) 
            except ValueError:
                continue 

    available_lots = list(set(available_lots))

    return available_lots

def get_parking_lots_main(excel_path, time, weekend_bool, parking_pass):
    weekday_times = pd.read_excel(excel_path, sheet_name="Weekday Times") 
    weekend_times = pd.read_excel(excel_path, sheet_name="Weekend Times") 

    if weekend_bool:
        weekend_times_split = pd.DataFrame()
        for col in weekend_times.columns:
            split_columns = weekend_times[col].apply(split_time_range)
            split_columns.columns = [f'{col}_start', f'{col}_end']
            weekend_times_split = pd.concat([weekend_times_split, split_columns], axis=1)
        return get_parking_lots_single(weekend_times_split, time, parking_pass)
    else:
        weekday_times_split = pd.DataFrame()
        for col in weekday_times.columns:
            split_columns = weekday_times[col].apply(split_time_range)
            split_columns.columns = [f'{col}_start', f'{col}_end']
            weekday_times_split = pd.concat([weekday_times_split, split_columns], axis=1)
        return get_parking_lots_single(weekday_times_split, time, parking_pass)
    
def create_coordinates_dict(excel_path):
    df = pd.read_excel(excel_path, sheet_name="Coordinates")
    
    coordinates_dict = {}
    for _, row in df.iterrows():
        lot_name = row.iloc[0]  
        lat = row["º N"]
        long = row["º W"]
        coordinates_dict[lot_name] = [lat, long]
    
    return coordinates_dict

def filter_coordinates_by_keys(coordinates_dict, keys_to_include):
    filtered_dict = {key: coordinates_dict[key] for key in keys_to_include if key in coordinates_dict}
    return filtered_dict

def get_parking_coordinate_dict(curr_time, weekend_bool, pass_name, file_name="../data/UK_Parking_Coords.xlsx"):
    return filter_coordinates_by_keys(create_coordinates_dict(file_name), get_parking_lots_main(file_name, curr_time, weekend_bool, pass_name))

In [16]:
get_parking_coordinate_dict(1700, True, 'Core')

{'PS2': [38.033587, 84.503843],
 'College Way W': [38.02084, 84.50286],
 'Sports Center Garage': [38.027316, 84.501604],
 'KF Red': [38.025202, 84.505422],
 'PS6': [38.034567, 84.511989],
 'Shively': [38.02854, 84.49908],
 'Scott Street Area': [38.04012, 84.50831],
 'College Way E': [38.021, 84.50162],
 'Complex Drive': [38.02819, 84.50259],
 'KF Blue': [38.02154, 84.507072],
 'KF Orange': [38.023525, 84.509702],
 'Memorial Coliseum': [38.03999, 84.49929],
 'Sport Center Drive': [38.02744, 38.02744],
 'PS1': [38.027366, 84.506248],
 'Wildcat Court': [38.01979, 84.50263],
 'Green': [38.023788, 84.503692],
 'Columbia Ave': [38.03152, 84.49831],
 'Commonwealth Drive': [38.01713, 84.50823],
 'North Red': [38.02501, 84.50575],
 'E7': [38.0274, 84.50135],
 'Large Gluck': [38.02525, 84.51015],
 'E28': [38.03764, 84.50777],
 'E27': [38.0348, 84.51047],
 'South Broadway': [38.04309, 84.50794],
 'E2': [38.03394, 84.50395],
 'PS5': [38.040771, 84.505187],
 'KF Periphery': [38.02507, 84.50543]}